In [2]:
import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import pandas_path  # Path style access for pandas
from tqdm import tqdm

import os

import torch                    
import torchvision
import fasttext

In [34]:
path = os.getcwd()
df = pd.read_json(path + '/MMHS150K/MMHS150K_GT.json', orient='index')

In [35]:
# labels: 0 - NotHate, 1 - Racist, 2 - Sexist, 3 - Homophobe, 4 - Religion, 5 - OtherHate

# extract tweet id from tweet url
df['tweet_id'] = df['tweet_url'].apply(lambda x: x.split('/')[-1])

# create a new column for each label
df['is_hateful'] = df['labels'].apply(lambda x: bool(x))
# create a new column for each label
df['is_racist'] = df['labels'].apply(lambda x: 1 in x)
df['is_sexist'] = df['labels'].apply(lambda x: 2 in x)
df['is_homophobe'] = df['labels'].apply(lambda x: 3 in x)
df['is_religion'] = df['labels'].apply(lambda x: 4 in x)
df['is_other_hate'] = df['labels'].apply(lambda x: 5 in x)

# remove the labels column
df = df.drop(columns=['labels'])
# remove the labels_str column
df = df.drop(columns=['labels_str'])

# reset index => remove date and get a new index
df = df.reset_index(drop=True)

In [36]:
df.head()

,img_url,tweet_url,tweet_text,tweet_id,is_hateful,is_racist,is_sexist,is_homophobe,is_religion,is_other_hate
0,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,1114679353714016256,True,True,False,True,True,False
1,http://pbs.twimg.com/ext_tw_video_thumb/106301...,https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,1063020048816660480,True,False,False,False,False,True
2,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,1108927368075374593,True,False,False,False,False,False
3,http://pbs.twimg.com/ext_tw_video_thumb/111401...,https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,1114558534635618305,True,True,False,False,False,False
4,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,1035252480215592966,True,True,False,False,False,False


In [32]:
# for each row, create the sample with the image from img_resized and the image text form img_txt
df["img_url"][0]

'http://pbs.twimg.com/tweet_video_thumb/D3gi9MHWAAAgfl7.jpg'

Input Representation:

- For the text: Use word embeddings like Word2Vec, GloVe, or FastText to represent each word in the tweet text.
- For the meme: Convert the image into a format that the model can understand, such as pixel values or feature vectors extracted using techniques like Convolutional Neural Networks (CNNs) or pre-trained models like VGG, ResNet, or Inception.